# Tensor API

In [8]:
import torch

a = torch.ones(3, 2)
a.shape
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [9]:
a_t = torch.transpose(a, 0, 1)
a_t

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [10]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [11]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

In [12]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

There is no difference between the two forms; they can be used interchangeably. We mentioned the online docs earlier (http://pytorch.org/docs). They are exhaustive and well organized, with the tensor operations divided into groups:
* `Creation ops`—Functions for constructing a tensor, like `ones` and `from_numpy`
* `Indexing, slicing, joining, mutating ops`—Functions for changing the shape, stride, or content of a tensor, like `transpose` 
* `Math ops`—Functions for manipulating the content of the tensor through computations